This notebook presents an application of the [pydiva2D](./pydiva2d.py) module to perform a 2D Diva analysis and then generate figures for the different input and output using the Basemap module.

In [ ]:
import os
import logging
import pydiva2d
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.basemap import Basemap
import subprocess
from importlib import reload
%matplotlib inline

Suppress mpl_toolkits warnings due to version mismatch:

In [ ]:
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)

If needed, modify the logger level to control the messages displayed on screen (by default: warnings).

In [ ]:
logger = logging.getLogger('pydiva2d')
logger.handlers[1].setLevel(logging.DEBUG)

# Prepare input files and directories

In this example the input files are already created and can be found in the *data* directory.

In [ ]:
datadir = './data/'
datafile = os.path.join(datadir, 'MLD1.dat')
coastfile = os.path.join(datadir, 'coast.cont')
paramfile = os.path.join(datadir, 'param.par')

We also create a directory *figures* where we will save the plots.

In [ ]:
figdir = './figures4github/'
if os.path.exists(figdir):
    logger.debug("Directory {0} already exists".format(figdir))
else:
    logger.debug("Creating directory {0}".format(figdir))
    os.makedirs(figdir)

Make figures and font size bigger; increase figure resolution:

In [ ]:
rcParams.update({'font.size': 16, 'figure.figsize': (10, 8), 'figure.dpi': 300})

## Create paths and file names

We create the paths for the Diva directories and files:

In [ ]:
divadir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"
DivaDirs = pydiva2d.DivaDirectories(divadir)
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

## Read input files

Let's read the information from the input files.<br>
For each input file, we first creat an object (i.e., **Data**, **Contour** and **Param**) that will allow us to perform operations such as 
* reading, 
* writing, 
* plotting, 
* getting a description.

In [ ]:
Data = pydiva2d.Diva2DData()
Data.read_from(datafile)
Contour = pydiva2d.Diva2DContours()
Contour.read_from(coastfile)
Param = pydiva2d.Diva2DParameters()
Param.read_from(paramfile)

For example with the parameters:

In [ ]:
Param.describe()

# Make the analysis

We write the inputs to the corresponding files using the *write_to* functions available for each input objects:

In [ ]:
Data.write_to(DivaFiles.data)
Param.write_to(DivaFiles.parameter)
Contour.write_to(DivaFiles.contour)

## Generate the mesh

We simply call the **divamesh** script:

In [ ]:
subprocess.Popen("./divamesh", cwd=DivaDirs.diva2d, shell=True)

and we create a new **Mesh** object using the information from the 2 files generated by the mesh generation:

In [ ]:
Mesh = pydiva2d.Diva2DMesh()
Mesh.read_from(DivaFiles.mesh, DivaFiles.meshtopo)

Similarly to the parameter, we can obtain a description of the mesh:

In [ ]:
Mesh.describe()

## Perform the analysis

We call the **divacalc** script:

In [ ]:
subprocess.Popen("./divacalc", cwd=DivaDirs.diva2d, shell=True)

and load the results in a new object **Analysis**:

In [ ]:
Analysis = pydiva2d.Diva2DResults(DivaFiles.result)

# Create the plots

In this example the plots are created using the Basemap module.<br>
All the objects we created have a function *add_to_plot* that takes into account, if it is defined, the selected projection in Basemap.

## Initialise the projection

We will center the map on the region specified by the parameters and with a Mercator projection.<br>
Type `Basemap?` for a list of available projections and options.

In [ ]:
m = Basemap(projection='merc', llcrnrlon=Param.xori, llcrnrlat=Param.yori, urcrnrlon=Param.xend,
            urcrnrlat=Param.yend, resolution='i')

## Data points

We use the function **add_to_plot** with 'm' (the projection) as an optional argument.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
m.ax = ax
Data.add_to_plot(m=m, s=10)
m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
m.drawcoastlines(linewidth=0.2)
plt.title('Data points')
plt.savefig(os.path.join(figdir, 'datapoints.png'))
plt.show()
plt.close()

## Contours

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
m.ax = ax
Contour.add_to_plot(m=m, linewidth=3, color='.5')
m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
m.drawcoastlines(linewidth=0.2, zorder=3)
plt.title('Contours')
plt.savefig(os.path.join(figdir, 'contours.png'))
plt.show()
plt.close()

## Finite element mesh

To create the plot, we add the option *m* in the **add_to_plot** function.<br>
Take into account that this plot generation is relatively slow.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
m.ax = ax
Mesh.add_to_plot(m, linewidth=0.25, color='k')
m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
m.drawcoastlines(linewidth=0.2, zorder=3)
plt.title("Finite-element mesh")
plt.savefig(os.path.join(figdir, 'mesh.png'))
plt.show()
plt.close()

## Analysed field

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
FieldPlot = Analysis.add_to_plot(ax=ax, m=m)
m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
m.drawcoastlines(linewidth=0.2, zorder=3)
plt.colorbar(FieldPlot, label="ok")
plt.title('Analyzed field')
plt.savefig(os.path.join(figdir, 'analysis.png'))
plt.show()
plt.close()

# Final remarks

* The different elements of the plot can be combined in many ways (see example below).
* All the plots can be easily modified by adapting the diffent options: colors, projection, font size etc.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
Analysis.add_to_plot(ax=ax, m=m, vmin=-60, vmax=-5)
Mesh.add_to_plot(ax=ax, m=m, alpha=0.8, linewidth=0.25, color='k')
Contour.add_to_plot(m=m, linewidth=1, color='0.75')
Data.add_to_plot(m=m, s=3, vmin=-60, vmax=-5)
m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.26)
m.drawcoastlines(linewidth=0.2, zorder=3)
plt.title('Analyzed field, mesh,\n contour and data')
plt.show()
plt.savefig(os.path.join(figdir, 'AnalysisMeshData.png'))
plt.close()